In [3]:
from typing import List
 
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import math
import torch
from torch.utils.data import Dataset, DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


### Installing Stanza

Installing and importing Stanza are as simple as running the following commands:

In [4]:
# Install stanza
%%capture
!pip install stanza

# Import stanza
import stanza
stanza.download('en')

### Setting up Stanford CoreNLP


In [5]:
# Download the Stanford CoreNLP package with Stanza's installation command
# This'll take several minutes, depending on the network speed
corenlp_dir = './corenlp'
stanza.install_corenlp(dir=corenlp_dir)

# Set the CORENLP_HOME environment variable to point to the installation location
import os
os.environ["CORENLP_HOME"] = corenlp_dir

## 2. Load CoreNLP Interface

### Constructing CoreNLPClient



In [6]:
# Import client module
from stanza.server import CoreNLPClient

In [7]:
# Construct a CoreNLPClient with some basic annotators, a memory allocation of 4GB, and port number 9001
client = CoreNLPClient(
    annotators=['dcoref'], 
    memory='6G', 
    endpoint='http://localhost:9001',
    outputFormat="json",
    
    be_quiet=False)
print(client)

# Start the background server and wait for some time
# Note that in practice this is totally optional, as by default the server will be started when the first annotation is performed
client.start()
import time; time.sleep(10)

In [8]:
# Initialize tokenizer used by dcoref model
nlp = stanza.Pipeline(lang='en', processors='tokenize', use_gpu= True)

# Load ECB+ Data

In [9]:
import random
import json
import pandas as pd
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

# Make sure to click "Add shortcut to drive" for the "Coref-for-GPT" folder
gdrive_dir_path = "/content/drive/MyDrive/Coref-for-GPT"

Mounted at /content/drive


In [10]:
# Change this path to the root dir where you stored data
root_path = gdrive_dir_path

In [11]:
# Path to the ecb data
ecb_path = f"{root_path}/Data/ECB+/processed/"

In [12]:
file_path = ecb_path + "dev_with_new_index.json"
with open(file_path) as f:
    dev = json.load(f)
print(len(dev))

192


# Parse and Match Data

In [13]:
# Parse gold mentions to get corresponding clusters
def get_gold_clusters(gold_mentions):
    clusters = {}
    for m_id in gold_mentions:
        mention = gold_mentions[m_id]
        cluster_id = mention["cluster_id"]
        if cluster_id in clusters:
            clusters[cluster_id].append(m_id)
        else:
            clusters[cluster_id] = [m_id]
    return clusters

In [14]:
# Parse annotation results from CoreNLP
def get_vocab(annotated_doc):
    vocab = [] 
    sents_divider = []
    idx = 0
    for sent in annotated_doc.sentence:
        for tok in sent.token:
            vocab.append(tok.word)
            idx += 1
        sents_divider.append(idx)
    return vocab, sents_divider

def get_sentence_id(idx, sents_divider):
    # idx starts from 0
    for i, divider in enumerate(sents_divider):
        if idx < divider:
            return i
    raise Exception(f"Error: {idx} not covered by {sents_divider}")

def get_pred_mentions_and_clusters(annotated_doc, vocab, sents_divider, map):
    mentions = {}
    clusters = {}
    
    for chain in annotated_doc.corefChain:
        cluster_id = chain.chainID
        mentions_in_chain = chain.mention
        
        mention_ids_in_cluster = []

        for m in mentions_in_chain:
            s_id = m.sentenceIndex 
            if s_id > 0:
                start = sents_divider[s_id-1]
            else:
                start = 0
            
            m_id = str([start+m.beginIndex, start+m.endIndex])
            tokens_ids = list(range(start+m.beginIndex, start+m.endIndex))
            
            tokens = []
            gold_tokens_ids = []
            for t_id in tokens_ids:
                tokens.append(vocab[t_id])
                gold_tok_id = map[t_id]
                gold_tokens_ids.append(gold_tok_id)

            mentions[m_id] = {"sentence_id": s_id , 
                              "tokens": tokens,
                              "tokens_ids": tokens_ids,
                              "gold_tokens_ids": gold_tokens_ids,
                              }
            mention_ids_in_cluster.append(m_id)

        clusters[cluster_id] = mention_ids_in_cluster

    return mentions,clusters


In [15]:
# Match tokens from CoreNLP with original gold tokens
def get_token_from_nlp(doc):
    tokens = []
    for i, sentence in enumerate(doc.sentences):
        tokens += [token.text for token in sentence.tokens]
    return tokens

def reproduce_tokens_from_gold(gold_tokens, vocab_set, nlp):
    new_pred_vocab = []
    map_pred_to_gold = []
    for i, token in enumerate(gold_tokens):
        if token in vocab_set:
            new_pred_vocab.append(token)
            map_pred_to_gold.append(i)
        else:
            # CoreNLP tokenizer might split original token into multiple tokens
            doc = nlp(token)
            new_tokens = get_token_from_nlp(doc)
            if new_tokens:
                new_pred_vocab += new_tokens
                map_pred_to_gold += [i]*len(new_tokens)
    return new_pred_vocab, map_pred_to_gold

# Sanity Check: make sure the reproduce tokens match the CoreNLP tokens
def check_same_vocabs(new_vocab, old_vocab):
    assert len(new_vocab) == len(old_vocab), "Error: Not in the same length."
    for i, tok in enumerate(new_vocab):
        assert tok == old_vocab[i], f"Error: {tok} doesn't match {old_vocab[i]} at {i}."
    return True

In [16]:
# Match predicted mentions with gold mentions, ignoring mentions that are not gold
def match(m_token_ids, gold_mentions, threshold):
    m_token_ids = set(m_token_ids)
    for m_id in gold_mentions:
        mention = gold_mentions[m_id]
        gold_token_ids = set(mention["tokens_ids"])
        intersec = gold_token_ids.intersection(m_token_ids)
        matched_ratio = len(intersec)/len(gold_token_ids)
        if matched_ratio > threshold:
            return m_id
    return -1

def matched_gold_mentions(mentions, gold_mentions, threshold = 0.5):
    mention_map = {}
    matched_mentions = []
    for m_id in mentions:
        m = mentions[m_id]
        matched_gold_m_id = match(m["gold_tokens_ids"], gold_mentions, threshold)
        mention_map[m_id] = matched_gold_m_id
        if matched_gold_m_id != -1:
            matched_mentions.append(matched_gold_m_id)
    return mention_map, matched_mentions

# Build clusters based on cluster predictions of gold mentions
def pred_clusters_with_gold_mentions(pred_clusters, mention_map):
    new_clusters = {}
    for i, cluster in pred_clusters.items():
        gold_mentions_in_cluster = []
        for m_id in cluster:
            if mention_map[m_id] != -1:
                gold_mentions_in_cluster.append(mention_map[m_id])
        if gold_mentions_in_cluster:
            new_clusters[i] = gold_mentions_in_cluster
    return new_clusters

In [17]:
# # Test case
# doc_name = "35_10ecb.xml"
# dev[doc_name][]

In [18]:
# # Test case
# doc_name = "35_10ecb.xml"
# text, gold_toks, gold_mentions, gold_sents_divider = dev[doc_name]
# gold_clusters = get_gold_clusters(gold_mentions)
# annotated_doc = client.annotate(" ".join(gold_toks))

In [19]:
# pred_vocab, pred_sents_divider = get_vocab(annotated_doc)
# pred_vocab_set = set(pred_vocab)
# new_vocab, map_pred_to_gold = reproduce_tokens_from_gold(gold_toks, pred_vocab_set, nlp)
# check_same_vocabs(new_vocab, pred_vocab)

In [20]:
# pred_mentions, pred_clusters = get_pred_mentions_and_clusters(annotated_doc, pred_vocab, pred_sents_divider, map_pred_to_gold)
# mention_map, matched_mentions = matched_gold_mentions(pred_mentions, gold_mentions, threshold = 0.5)
# print(len(gold_mentions), len(matched_mentions))

# Generate Pairwise Annotation for Gold Mentions

In [21]:
def get_pairwise_labels(mentions_ids, clusters):
    pairs = []
    labels = []
    for i in range(len(mentions_ids)-1):
        for j in range(i+1, len(mentions_ids)):
            m1_id = mentions_ids[i]
            m2_id = mentions_ids[j]
            pair = set([m1_id, m2_id])
            label = 0
            for cluster_mentions in clusters.values():
                if pair.issubset(set(cluster_mentions)):
                    label = 1
                    break
            pairs.append(pair)
            labels.append(label)
            
    pairs, labels = np.array(pairs), np.array(labels)
    return pairs, labels

In [22]:
# # Test case
# pred_clusters_gold_mentions = pred_clusters_with_gold_mentions(pred_clusters, mention_map)
# pairs, labels = get_pairwise_labels(list(gold_mentions.keys()), gold_clusters)
# _, predictions = get_pairwise_labels(list(gold_mentions.keys()), pred_clusters_gold_mentions)

# print("Labels: ", labels)
# print("Preds: ", predictions)

In [23]:
def get_result(gold_clusters, pred_clusters, gold_mentions, text, sentence_restriction = 2):
    # gold
    gold_pairs, gold_labels = get_pairwise_labels(list(gold_mentions.keys()), gold_clusters)
    gold_sents = text.split("[EOS]")
    df = pd.DataFrame([gold_pairs, gold_labels]).T
    df.columns = ["pair", "label"]

    # predictions
    _, pred_labels = get_pairwise_labels(list(gold_mentions.keys()), pred_clusters)
    df["pred"] = pred_labels
    
    # pair information
    df["mention_pair"] = df["pair"].apply(lambda x: [gold_mentions[list(x)[0]], gold_mentions[list(x)[1]]])
    df["pair"] = df["pair"].astype(str)

    # sentence_restriction=2: only keep pairs from consecutive sentences
    df["sent_idx"] = df["mention_pair"].apply(lambda x: [x[0]["sentence_id"], x[1]["sentence_id"]])
    df["sentence"] = df["sent_idx"].apply(lambda x: [gold_sents[x[0]],gold_sents[x[1]]])
    df["sent_filter"] = df["sent_idx"].apply(lambda x: np.abs(x[1]-x[0]))
    if sentence_restriction:
        df = df[df["sent_filter"] < sentence_restriction].reset_index(drop = True)
 
    return df

# Annotate Data 

In [24]:
def annotate(data, matching_threshold = 0.5):
    n_skipped = 0
    for doc_name in tqdm(data):
        print(f"Parsing ----> {doc_name}")
        text, gold_toks, gold_mentions, _, = data[doc_name]
        gold_clusters = get_gold_clusters(gold_mentions)
        try:
            annotated_doc = client.annotate(" ".join(gold_toks))
        except:
            print(f"Skip (client issue) ----> {doc_name}")
            continue
        pred_vocab, pred_sents_divider = get_vocab(annotated_doc)
        pred_vocab_set = set(pred_vocab)
        new_vocab, map_pred_to_gold = reproduce_tokens_from_gold(gold_toks, pred_vocab_set, nlp)
        try:
            check_same_vocabs(new_vocab, pred_vocab)
        except:
            print("Not matching: ", new_vocab, pred_vocab)
            print(f"Skip ----> {doc_name}")
            n_skipped += 1
            continue
        
        pred_mentions, pred_clusters = get_pred_mentions_and_clusters(annotated_doc, pred_vocab, pred_sents_divider, map_pred_to_gold)
        mention_map, matched_mentions = matched_gold_mentions(pred_mentions, gold_mentions, threshold = matching_threshold)

        pred_clusters_gold_mentions = pred_clusters_with_gold_mentions(pred_clusters, mention_map)
        result = get_result(gold_clusters, pred_clusters_gold_mentions, gold_mentions, text, sentence_restriction = 2)
        result["doc_name"] = doc_name

        output_file = f"{root_path}/Results/Multi-pass-Sieve/pairwise_result.csv"
        result.to_csv(output_file, mode="a", index=False, header=None)

    columns = result.columns
    
    print(f"Files saved with headers {columns}")
    return n_skipped

In [25]:
n_skipped = annotate(dev)
print(f"skipped {n_skipped} in total")

  0%|          | 0/192 [00:00<?, ?it/s]

Parsing ----> 35_10ecb.xml


  1%|          | 1/192 [00:31<1:40:12, 31.48s/it]

Parsing ----> 35_11ecbplus.xml


  1%|          | 2/192 [00:38<54:47, 17.30s/it]  

...

Skip (client issue) ----> 12_6ecbplus.xml
Parsing ----> 12_9ecb.xml


 99%|█████████▉| 191/192 [22:10<00:19, 19.40s/it]

Not matching:  ['The', 'Indian', 'navy', 'says', 'it', 'has', 'arrested', '23', 'Somali', 'and', 'Yemeni', 'pirates', 'who', 'tried', 'to', 'storm', 'a', 'ship', 'in', 'the', 'Gulf', 'of', 'Aden', '.', 'A', 'navy', 'spokesman', 'said', 'it', 'had', 'responded', 'to', 'a', 'mayday', 'call', 'from', 'MV', 'Gibe', ',', 'flying', 'under', 'the', 'Ethiopian', 'flag', '.', 'Several', 'countries', 'have', 'warships', 'patrolling', 'the', 'gulf', 'amid', 'growing', 'international', 'concern', 'about', 'piracy', '.', 'Meanwhile', ',', 'US', 'Defence', 'Secretary', 'Robert', 'Gates', 'said', 'better', 'intelligence', 'was', 'needed', 'for', 'a', 'land', 'attack', 'on', 'pirate', 'bases', 'to', 'be', 'considered', '.', 'Mr', 'Gates', ',', 'speaking', 'at', 'a', 'security', 'conference', 'in', 'Bahrain', ',', 'also', 'called', 'for', 'shipping', 'companies', 'to', 'do', 'more', 'to', 'protect', 'their', 'vessels', 'travelling', 'through', 'the', 'Arabian', 'Sean', 'and', 'Indian', 'Ocean', '.', 'T

100%|██████████| 192/192 [23:10<00:00,  7.24s/it]

Skip (client issue) ----> 12_9ecbplus.xml
Files saved with headers Index(['pair', 'label', 'pred', 'mention_pair', 'sent_idx', 'sentence',
       'sent_filter', 'doc_name'],
      dtype='object')
skipped 65 in total


# Shut down the server

In [27]:
# Shut down the background CoreNLP server
client.stop()

# time.sleep(10)
!ps -o pid,cmd | grep java

   2212 /bin/bash -c ps -o pid,cmd | grep java
   2214 grep java
